# Data Modeling

This notebook carries out basic data modeling of the [Dataset of songs in Spotify](https://www.kaggle.com/datasets/mrmorj/dataset-of-songs-in-spotify); then, this data modeling is trasferred to production-level code in the components placed in the uper level of this folder.

Note that the present notebook is not tracked with Weights and Biases. To see how to track notebooks, have a look at [`EDA_Tracked.ipynb`](EDA_Tracked.ipynb).